# Trabalho Prático 1 de Estruturas Criptográficas

  - **Autores:** (Grupo 9)
      - Nelson Faria (A84727)
      - Miguel Oliveira (A83819)

## Exercício 2

In [1]:
# Imports necessários à execução deste notebook
import os, hashlib

### Construir uma classe Python que implemente um KEM-RSA

A classe deve:
  - Inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits do módulo RSA) e gere as chaves pública e privada.
  - Conter funções para encapsulamento e revelação da chave gerada.

In [88]:
# Classe que implementa o KEM_RSA (Key Encapsulation Mechanism) baseado no RSA
# Baseamo-nos em: https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
#                 https://docs.python.org/3/library/hashlib.html

class KEM_RSA(object):
    
    def __init__(self, N, timeout=None):
        
        # Obter um primo aleatorio para o parametro q do rsa no intervalo [2^(N-1);2^N-1]
        self.q = self.primoAleatorio(N)
        # Obter um primo aleatorio para o parametro p do rsa no intervalo [2^(N+1-1);2^(N+1)-1]
        self.p = self.primoAleatorio(N + 1)
        # n = p * q
        self.n = self.p * self.q
        # Funcao totiente de euler = (p-1)*(q-1)
        self.phi = (self.p-1)*(self.q-1)
        # Escolher um inteiro aleatorio e entre 1 < e < φ(n) e que e e φ(n) sejam relativamente primos
        self.e = ZZ.random_element(self.phi)
        while gcd(self.e, self.phi) != 1:
            self.e = ZZ.random_element(self.phi)
        
        # Para calcular d, usamos o 'extended Euclidean algorithm': de−k⋅φ(n)=1 -> Assimm, so precisamos de descobrir d e -k
        # xgcd(x, y) retorna um triplo (g, s, t) que satisfaz a identidade de Bézout: g=gcd(x,y)=sx+ty 
        self.bezout = xgcd(self.e, self.phi)
        # d = mod(s,φ(n)), uma vez que 1 < d < φ(n)
        self.d = Integer(mod(self.bezout[1], self.phi))
        
          
    # Retorna um primo aleatorio entre 2^(N-1) e 2^N - 1
    def primoAleatorio(self, N):
        
        return random_prime(2**N-1,True,2**(N-1))

    
    # Funcao que serve para encapsular a chave que for acordada a partir de uma chave publica
    def encapsula(self, e, n):
        
        # Escolher um inteiro aleatorio r entre 0 < r < n 
        r = ZZ.random_element(n)
        
        # Criptograma com este inteiro usado para o encapsulamento da chave (c ← r^e mod n)
        c = Integer(power_mod(r, e, n))
        
        # Gerar o salt para derivar a chave
        salt = os.urandom(16)
        # Geracao da chave simetrica a partir do r (W ← KDF(r))
        w = hashlib.pbkdf2_hmac('sha256', str(r).encode(), salt, 100000)
        
        return (w, salt + str(c).encode())
    
    
    # Funcao usada para desencapsular uma chave, a partir do seu "encapsulamento"
    def desencapsula(self, cs):
        
        # Buscar os 16 primeiros bytes para obter o salt e o restante é o "encapsulamento" da chave
        salt = cs[:16]
        c = int(cs[16:].decode())
        
        # Obter o r (r ← c^d mod n) com o algoritmo power_mod
        r = Integer(power_mod(c, self.d, self.n))
        
        # Geracao da chave simetrica a partir do r (W ← KDF(r))
        w = hashlib.pbkdf2_hmac('sha256', str(r).encode(), salt, 100000)
        
        return w
        

**Testagem da classe definida acima:**

In [89]:
# Parametro de seguranca
N = 1024

# Chave publica: (n,e)
# Chave privada: (p,q,d)
# Inicializacao da classe responsavel por implementar o KEM-RSA
kemrsa = KEM_RSA(N)

# Verificar que ed == 1 (mod φ(n))
#print(mod(kemrsa.e * kemrsa.d, kemrsa.phi))

# Procede-se ao encapsulamento
(w,c) = kemrsa.encapsula(kemrsa.e, kemrsa.n)

print("Chave devolvida pelo encapsulamento: ")
print(w)
print("\n'Encapsulamento' da chave: ")
print(c)

# Procede-se ao desencapsulamento
w1 = kemrsa.desencapsula(c)
print("\nChave devolvida pelo desencapsulamento: ")
print(w1)

Chave devolvida pelo encapsulamento: 
b'>iN\n\xee:uNe\xcf;m\xfd\xec>\xad6\xde\x0b\xd0t7J\xb8\xd7?\'"\x06\x11\xb8\x1d'

'Encapsulamento' da chave: 
b'\xc22\x01T\xac\xfd\xc5QMo\xa1\x86@\x92l\xa710532067822825801805377844922718841480917297303947499157924271209537208771548687624569416118776305927616683626909878847639515377689778578285867366585750716734401255598642812782128377404073619552291508644675361899615525653135497696907454652151630436729540286692517934577502749906370784553044017732218538204864730152463505276131242532492426152481009000428480440793733759312344805523203377631505605785239638017573521704235236842850171312391504052957402780112114567192309640355860749202120098211905515024037064650110050989452689427637262685534263542945416771634698361698177829248487813500509498136253323660482615662748069'

Chave devolvida pelo desencapsulamento: 
b'>iN\n\xee:uNe\xcf;m\xfd\xec>\xad6\xde\x0b\xd0t7J\xb8\xd7?\'"\x06\x11\xb8\x1d'


### Construir,  a partir do KEM definido anteriormente e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.